In [1]:
import numpy as np
from numpy import random
from tqdm import tqdm
import h5py
import random
from sklearn import preprocessing
import seaborn as sns
import linear_optimizer5
import math
import matplotlib
import matplotlib.pyplot as plt
import yaml
import THzChannel1_02
import csv
from time import sleep

1.7.1
1.8


In [2]:
def distance_array_gen(distance_array_nums, nodes_num):
    # Mean = 0.8km, Sigma = 0.05km
    distance_array = np.random.normal(0.8, 0.05, size=(distance_array_nums, nodes_num-1))
    return distance_array

In [3]:
def THz_model(distance):
    node1_position = [0, 0]
    node2_position = [0, 0]
    d=0
    distance_example = []
    capacitance = []
    while(d <= 2.1): 
        cap = THzChannel1_02.find_channel_capacity(node1_position,node2_position)
        #print('channel capacitance at ',d,'km is ',cap,'Tbps')
        distance_example.append(d)
        capacitance.append(cap)
        node2_position[1] += 0.1
        d += 0.1
    #print('THz_model is',capacitance,'(Unit in Tbps)')
    distance = int(distance/0.1)
    return capacitance[distance]

In [4]:
def link_calculator(timestamp, source_node, distance_array, mobility_prediction_flag):
    distance = distance_array[int(timestamp),int(source_node-1)]
    
    # return realtime_link_capacity
    if 0 <= distance <= 2:
        THz_max = THz_model(0)
        link_regulation = THz_model(distance)/THz_max
        # Best
        if mobility_prediction_flag == 4:
            return link_regulation
        elif mobility_prediction_flag == 3:
            if source_node % 2 == 0:
                # print('9999999999999')
                return link_regulation
            return 0.81 * link_regulation
        # Middium
        elif mobility_prediction_flag == 2:
            if source_node % 2 == 0:
#                 print('9999999999999')
                return 0.81 * link_regulation
            return 0.64 * link_regulation
        # Worst
        elif mobility_prediction_flag == 1:
            if source_node % 2 == 0:
#                 print('9999999999999')
                return 0.64 * link_regulation
            return 0.25 * link_regulation
    
    else:
        print('************* Distance error at ', timestamp,' node', source_node, ' *************')
        return 0

In [5]:
def Delay_calc(nodes_num, dataloader_old, dataloader, delta_tau):
    # return delay per Tx_period
    diff = np.zeros(nodes_num)
    delay = 0
    for i in range(nodes_num-2):
        add_size = dataloader_old[i+1, 2] + dataloader[i+1, 2]
        add_two = dataloader_old[i+1, 1] + dataloader[i+1, 1]
        size = (add_size - add_two) / 2 
        delay += size / delta_tau
        diff[i+1] = size
    return [delay, diff]

In [6]:
def TXRX(source_node,dataloader,timestamp,link_capacity,sending_rate,mobility_prediction_flag, distance_array,diff):
    # 1. When align, PLR < 3%
    # 2. d, align will influence PLR and link capacity(block)
    # 3. buffer influence, from link and noise
    # 4. sending rate and link rate
    
    [Tx_wait_tx, Tx_bufferremain, Tx_buffersize] = dataloader[source_node,:]
    [Rx_wait_tx, Rx_bufferremain, Rx_buffersize] = dataloader[source_node+1,:]
    tmp = 0
    # Cauculate Ideal Link rate -- Distance and Alignment mode
    link_rate = link_capacity[source_node] * link_calculator(timestamp, source_node, distance_array, mobility_prediction_flag)

    # No extend
    if source_node == 0:
        link_rate = min(sending_rate, link_rate)
    # Tx start
    data_txed = min(link_rate, Tx_wait_tx)    
    # No extend
    tx_delay = diff[source_node] / link_rate
    data_rxed = min(Rx_bufferremain, data_txed)
    
    # Update dataloader
    dataloader[source_node,0] -= data_txed
    if source_node != 0:
        dataloader[source_node,1] += data_txed
    dataloader[source_node+1,0] += data_rxed
    dataloader[source_node+1,1] = Rx_bufferremain - data_rxed
    
    # Update data_loss
    tmp += data_txed - data_rxed
                    
    return [dataloader, tmp, tx_delay]

In [7]:
def main(sending_rate, mobility_prediction_flag):
    random.seed()
    timestamp = 0
    buffer_obstruct_row = 0
    odd_flag = True
    loss = 0 # bit
    tru = 0 # bps
    delay = 0
    count = 1 # Count how many times tru be added 
    diff = np.zeros(nodes_num)

    # Create distance array
    distance_array = distance_array_gen(distance_array_nums,nodes_num) # [N_of_rows,N_of_links]

    # Initial dataloader
    # [data_waiting_trans, buffer_remain, buffer_size]
    dataloader = np.zeros([nodes_num,3])
    dataloader[:,1] = [buffersize_criterion * i for i in buffersize_coefficent]
    dataloader[:,2] = dataloader[:,1]
    dataloader[0,0] = dataloader[0,1] = dataloader[-1,1] = dataloader[0,2] = dataloader[-1,2] = datasize
    dataloader_delta = np.zeros(nodes_num)

    # loop ends on all nodes no need to Tx
    while np.any(dataloader[:-1,0]):  
        # 1. Check which nodes on
        # 2. Check datasize
        # 3. Return data transed, data received and buffer remain
        # 4. Switch odd even or not
        if odd_flag:
            dataloader_old = dataloader
            tru_old = dataloader_old[-1,0]
            for nodes in range(int(math.ceil((nodes_num-1)/2))):
                node = nodes*2
                delay_sum = 0
                for _ in range(int(Tx_period/mini_resolution)):
                    # No data need to trans then continue
                    if dataloader[node,0] <= 0: 
                        continue
                    # Check buffer obstruct and dataloader updated
#                     if buffer_obstruct_array[buffer_obstruct_row,0] == timestamp and buffer_obstruct_array[buffer_obstruct_row,1] == node+1:
#                         newbuffer = dataloader[node+1,1] - buffer_obstruct_array[buffer_obstruct_row,2]
#                         dataloader[node+1,1] = max(0,newbuffer)
#                         buffer_obstruct_row += 1
                    # TXRX
                    [dataloader, data_loss, tx_delay] = TXRX(node,
                             dataloader,
                             timestamp, 
                             link_capacity,
                             sending_rate,
                             mobility_prediction_flag,
                             distance_array,
                             diff)
                    # Calculate loss per mini_resolution
                    loss += data_loss
#                     print('tx_delay',tx_delay*mini_resolution)
                    delay_sum += tx_delay*mini_resolution
                delay += delay_sum/int(Tx_period/mini_resolution)
#                 print('sum',delay_sum/int(Tx_period/mini_resolution))
            # Tx next round and update timestamp
            timestamp += Tx_period
            timestamp = round(timestamp,3)
            # Calculate Troughput on last node
            if node == nodes_num - 2:
                tru_new = dataloader[-1,0] - tru_old
                tru += tru_new/Tx_period
                count += 1
            # Sum delay per Tx_period 2ms
            [in_delay, diff] = Delay_calc(nodes_num, dataloader_old, dataloader, delta_tau)
#             print('in_delay',in_delay)
            delay += in_delay
            # Update even odd flag
            odd_flag = False

        if not odd_flag:
            dataloader_old = dataloader
            tru_old = dataloader_old[-1,0]
            for nodes in range(int(math.ceil((nodes_num-1)/2))):
                node = nodes*2 + 1
                delay_sum = 0
                for _ in range(int(Tx_period/mini_resolution)):
                    # No data need to trans then continue
                    if dataloader[node,0] <= 0: 
                        continue
                    # Check buffer obstruct and dataloader updated
#                     if buffer_obstruct_array[buffer_obstruct_row,0] == timestamp and buffer_obstruct_array[buffer_obstruct_row,1] == node+1:
#                         newbuffer = dataloader[node+1,1] - buffer_obstruct_array[buffer_obstruct_row,2]
#                         dataloader[node+1,1] = max(0,newbuffer)
#                         buffer_obstruct_row += 1
                    # TXRX
                    [dataloader, data_loss, tx_delay] = TXRX(node,
                             dataloader,
                             timestamp, 
                             link_capacity,
                             sending_rate,
                             mobility_prediction_flag,
                             distance_array,
                             diff)
                    # Calculate loss per mini_resolution
                    loss += data_loss
#                     print('tx_delay',tx_delay*mini_resolution)
                    delay_sum += tx_delay*mini_resolution
                delay += delay_sum/int(Tx_period/mini_resolution)
#                 print('sum',delay_sum/int(Tx_period/mini_resolution))
            # Tx next round and update timestamp
            timestamp += Tx_period
            timestamp = round(timestamp,3)
            # Calculate Troughput on last node
            if node == nodes_num - 2:
                tru_new = dataloader[-1,0] - tru_old
                tru += tru_new/Tx_period
                count += 1
            # Sum delay per Tx_period 2ms
            [in_delay, diff] = Delay_calc(nodes_num, dataloader_old, dataloader, delta_tau)
#             print('in_delay',in_delay)
            delay += in_delay
            # Update even odd flag
            odd_flag = True

    return [timestamp, delay/count, loss, tru/count, dataloader]

In [8]:
# Insert hyperparameter from config
config = yaml.load(open('config.yml'), Loader=yaml.FullLoader)

nodes_num = config['nodes_num'] # 5 nodes
datasize = config['datasize'] # 1000 Gb
mini_resolution = config['mini_resolution'] # 0.1 ms
Tx_period = config['Tx_period'] # 2 ms

buffersize_criterion = config['buffersize_criterion'] # 0.1Gb
buffersize_coefficent = config['buffersize_coefficent'] # [1,1,1,1,1], buffersize for each node
delta_tau = config['delta_tau'] # Speed of trans in buffer

link_criterion = config['link_criterion'] # 100Gbps
link_criterion = link_criterion * mini_resolution # 10Mb/mini_resolution
link_coefficent = config['link_coefficent'] # [1,1,1,1] link_rate for each link

sending_rate = config['sending_rate'] # SR on first node, 80Gbps
sending_rate = sending_rate * mini_resolution # 8Mb/mini_resolution
batch_size = config['batch_size'] # 0.6Mb 
                                  # 10 batchs in 0.1 ms = 60 Gbps

# batch_per_second = config['batch_size'] # 4Mb/package
# batch_nums_per_resoluiton = sending_rate / batch_per_second # 20000 packages/s; 40 packages/2 ms; 2 packages/resolution

buffer_obstruct_nums = config['buffer_obstruct_nums'] # nums of buffer_noise
distance_array_nums = config['distance_array_nums'] # N of rows of distance
prediction_flag = config['prediction_flag'] # predcition algorithm mode
mobility_flag = config['mobility_flag'] # mobility mode

# Creat link and buffer
link_capacity = np.zeros(nodes_num-1)
link_capacity = [link_criterion * i for i in link_coefficent]
buffer_obstruct_array = np.zeros([buffer_obstruct_nums,3]) # [timestamp, buffer_obstruct_node, buffer_obstruct_size]


In [9]:
sending_rate_nums = 100
times_per_rate = 20
Delay0 = []
Delay1 = []
Delay2 = []
Delay3 = []
BER0 = []
BER1 = []
BER2 = []
BER3 = []
THR0 = []
THR1 = []
THR2 = []
THR3 = []

dataloader = []
main_flag = 1
ideal = True
mobility_prediction_flag = 4
for i in tqdm(range(sending_rate_nums)):
    D, L, T = 0, 0, 0
    for j in range(times_per_rate):
        if main_flag:
            [time, delay, loss, tru, dataloader] = main(sending_rate*(i+1)/sending_rate_nums, mobility_prediction_flag)
        else:
            [time, delay, loss, batchloader] = main_P(sending_rate*(i+1)/sending_rate_nums, mobility_prediction_flag)
        D += delay
        L += loss
        T += tru
    # Delay unit is ms/bit
    Delay0.append(D/times_per_rate)
    # Loss unit is percentage
    L = L/times_per_rate
    BER0.append(L*100/datasize)
    # Throughput unit is Gbps
    THR0.append(T/times_per_rate/1000000000)
    sleep(0.01)
sleep(0.5)

for i in tqdm(range(sending_rate_nums)):
    D, L, T = 0, 0, 0
    for j in range(times_per_rate):
        if main_flag:
            [time, delay, loss, tru, dataloader] = main(sending_rate*(i+1)/sending_rate_nums, mobility_prediction_flag-1)
        else:
            [time, delay, loss, batchloader] = main_P(sending_rate*(i+1)/sending_rate_nums, mobility_prediction_flag-1)
        D += delay
        L += loss
        T += tru
    # Delay unit is ms/bit
    Delay1.append(D/times_per_rate)
    # Loss unit is percentage
    L = L/times_per_rate
    BER1.append(L*100/datasize)
    # Throughput unit is Gbps
    THR1.append(T/times_per_rate/1000000000)
    sleep(0.01)
sleep(0.5)


for i in tqdm(range(sending_rate_nums)):
    D, L, T = 0, 0, 0
    for j in range(times_per_rate):
        if main_flag:
            [time, delay, loss, tru, dataloader] = main(sending_rate*(i+1)/sending_rate_nums, mobility_prediction_flag-2)
        else:
            [time, delay, loss, batchloader] = main_P(sending_rate*(i+1)/sending_rate_nums, mobility_prediction_flag-2)
        D += delay
        L += loss
        T += tru
    # Delay unit is ms/bit
    Delay2.append(D/times_per_rate)
    # Loss unit is percentage
    L = L/times_per_rate
    BER2.append(L*100/datasize)
    # Throughput unit is Gbps
    THR2.append(T/times_per_rate/1000000000)

    sleep(0.01)
sleep(0.5)


for i in tqdm(range(sending_rate_nums)):
    D, L, T = 0, 0, 0
    for j in range(times_per_rate):
        if main_flag:
            [time, delay, loss, tru, dataloader] = main(sending_rate*(i+1)/sending_rate_nums, mobility_prediction_flag-3)
        else:
            [time, delay, loss, batchloader] = main_P(sending_rate*(i+1)/sending_rate_nums, mobility_prediction_flag-3)
        D += delay
        L += loss
        T += tru
    # Delay unit is ms/bit
    Delay3.append(D/times_per_rate)
    # Loss unit is percentage
    L = L/times_per_rate
    BER3.append(L*100/datasize)
    # Throughput unit is Gbps
    THR3.append(T/times_per_rate/1000000000)

    sleep(0.01)
sleep(0.5)


100%|███████████████████████████████████████| 100/100 [2:12:09<00:00, 79.30s/it]


In [10]:
x = np.arange(0, sending_rate/100000, sending_rate/sending_rate_nums/100000)        
x = x + sending_rate/sending_rate_nums/100000    
x = x.tolist()
x.insert(0,0)
print('Sending Rate', x)
Delay0.insert(0, 0)
Delay1.insert(0, 0)
Delay2.insert(0, 0)
Delay3.insert(0, 0)

BER0.insert(0, 0)
BER1.insert(0, 0)
BER2.insert(0, 0)
BER3.insert(0, 0)

THR0.insert(0, 0)
THR1.insert(0, 0)
THR2.insert(0, 0)
THR3.insert(0, 0)

print('Delay0:', Delay0, '\nDelay1:', Delay1, '\nDelay2:', Delay2, '\nDelay3:', Delay3, '\n\n')
print('BER0:', BER0, '\nBER1:', BER1,  '\nBER2:', BER2, '\nBER3:', BER3, '\n\n')
print('THR0', THR0, '\nTHR1', THR1,  '\nTHR2', THR2, '\nTHR3:', THR3)

Sending Rate [0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0, 100.0]
Delay0: [0, 7.33351938921805e-06, 1.466014744553617e-05, 2.1965306589400678e-05, 4.671329217520912e-05, 5.82578202161668e-05, 6.992124674368798e-05, 9.75333634385654e-05, 0.00012715570804033591, 0.00014320958945537307, 0.000160927132264904, 0.00022263101224945266, 0.00024256567746254215, 0.0002642538471629754, 0.00031226451552718423, 0.0003664595835383765, 0.0003906173114146818, 0.0004613

In [1]:
plt.figure(1)
fig, ax = plt.subplots()  # Create a figure and an axes.
ax.plot(x, Delay0, label='Prediction & low mobility (ideal)', color = 'black', marker = '*', 
        markerfacecolor='blue', markersize=6)
ax.plot(x, Delay1, label='Prediction & low mobility with interfere', color = 'blue', marker = 'o', 
        markerfacecolor='blue', markersize=6)
ax.plot(x, Delay2, label='Unprediction & low mobility', color = 'green', marker = '+', 
        markerfacecolor='green', markersize=6)
ax.plot(x, Delay3, label='Unprediction & high mobility', color = 'red', marker = 'X', 
        markerfacecolor='red', markersize=6)

ax.set_xlabel('Sending Rate (Gbps)')
ax.set_ylabel('Delays (s)')
ax.set_title("Delay over sending rate with mixed situations")
ax.axis([-5, 105, 0, 0.05])
ax.legend()

plt.figure(2)
fig, ax = plt.subplots()  # Create a figure and an axes.
ax.plot(x, BER0, label='Prediction & low mobility (ideal)', color = 'black', marker = '*', 
        markerfacecolor='blue', markersize=6)
ax.plot(x, BER1, label='Prediction & low mobility with interfere', color = 'blue', marker = 'o', 
        markerfacecolor='blue', markersize=6)
ax.plot(x, BER2, label='Unprediction & low mobility', color = 'green', marker = '+', 
        markerfacecolor='green', markersize=6)
ax.plot(x, BER3, label='Unprediction & high mobility', color = 'red', marker = 'X', 
        markerfacecolor='red', markersize=6)

ax.set_xlabel('Sending Rate (Gbps)')
ax.set_ylabel('Bit Error Rates (%)')
ax.set_title("BER over sending rate with mixed situations")
ax.axis([-5, 105, 0, 80])
ax.legend()

plt.figure(3)
fig, ax = plt.subplots()  # Create a figure and an axes.
ax.plot(x, THR0/2, label='Prediction & low mobility (ideal)', color = 'black', marker = '*', 
        markerfacecolor='blue', markersize=6)
ax.plot(x, THR1/2, label='Prediction & low mobility with interfere', color = 'blue', marker = 'o', 
        markerfacecolor='blue', markersize=6)
ax.plot(x, THR2/2, label='Unprediction & low mobility', color = 'green', marker = '+', 
        markerfacecolor='green', markersize=6)
ax.plot(x, THR3/2, label='Unprediction & high mobility', color = 'red', marker = 'X', 
        markerfacecolor='red', markersize=6)

ax.set_xlabel('Sending Rate (Gbps)')
ax.set_ylabel('Throughputs (Gbps)')
ax.set_title("throughput over sending rate with mixed situations")
ax.axis([-5, 105, 0, 80])
ax.legend()

plt.show()

NameError: name 'plt' is not defined

In [13]:
with open('./output/Delay0.csv', 'w', ) as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    for word in Delay0:
        wr.writerow([word])

with open('./output/Delay1.csv', 'w', ) as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    for word in Delay1:
        wr.writerow([word])

with open('./output/Delay2.csv', 'w', ) as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    for word in Delay2:
        wr.writerow([word])
        
with open('./output/Delay3.csv', 'w', ) as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    for word in Delay3:
        wr.writerow([word])
        
        
with open('./output/BER0.csv', 'w', ) as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    for word in BER0:
        wr.writerow([word])

with open('./output/BER1.csv', 'w', ) as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    for word in BER1:
        wr.writerow([word])
        
with open('./output/BER2.csv', 'w', ) as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    for word in BER2:
        wr.writerow([word])
        
with open('./output/BER3.csv', 'w', ) as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    for word in BER3:
        wr.writerow([word])
        

with open('./output/THR0.csv', 'w', ) as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    for word in THR0:
        wr.writerow([word])

with open('./output/THR1.csv', 'w', ) as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    for word in THR1:
        wr.writerow([word])
        
with open('./output/THR2.csv', 'w', ) as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    for word in THR2:
        wr.writerow([word])
        
with open('./output/THR3.csv', 'w', ) as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    for word in THR3:
        wr.writerow([word])